In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/1_INEL/Inel_ML/amazon_data.csv')
data_test = pd.read_csv('/content/drive/MyDrive/1_INEL/Inel_ML/amazon_data2.csv')

In [ ]:
data_train['label']=data_train['stars'].map(lambda x: 1 if x > 3 else 0)
data_test['label']=data_test['stars'].map(lambda x: 1 if x > 3 else 0)

In [ ]:
x_train = data_train['review_body'].to_list()
y_train = data_train['label'].to_list()

x_test = data_test['review_body'].to_list()
y_test = data_test['label'].to_list()

In [ ]:
import re
def cleanText(text):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = whitespace.sub(' ', text)
    text = web_address.sub('', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]", "", text)
    text = re.sub("\d+", "", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    return text.lower()

In [ ]:
x_train = [cleanText(txt) for txt in x_train]
x_test = [cleanText(txt) for txt in x_test]

In [ ]:
max_sequence_len = max([len(x) for x in x_train])

In [ ]:
max_sequence_len

2981

In [ ]:
encoder = tf.keras.layers.TextVectorization(
 max_tokens=5000,
 output_mode='int',
 output_sequence_length=max_sequence_len)

In [ ]:
encoder.adapt(x_train)

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train, epochs=10)

Epoch 1/10
6250/6250 [==============================] - 290s 44ms/step - loss: 0.3931 - accuracy: 0.8172
Epoch 2/10
6250/6250 [==============================] - 224s 36ms/step - loss: 0.3467 - accuracy: 0.8474
Epoch 3/10
6250/6250 [==============================] - 223s 36ms/step - loss: 0.3296 - accuracy: 0.8546
Epoch 4/10
6250/6250 [==============================] - 220s 35ms/step - loss: 0.3135 - accuracy: 0.8616
Epoch 5/10
6250/6250 [==============================] - 217s 35ms/step - loss: 0.3001 - accuracy: 0.8677
Epoch 6/10
6250/6250 [==============================] - 220s 35ms/step - loss: 0.2886 - accuracy: 0.8734
Epoch 7/10
6250/6250 [==============================] - 227s 36ms/step - loss: 0.2781 - accuracy: 0.8789
Epoch 8/10
6250/6250 [==============================] - 229s 37ms/step - loss: 0.2680 - accuracy: 0.8835
Epoch 9/10
6250/6250 [==============================] - 230s 37ms/step - loss: 0.2589 - accuracy: 0.8875
Epoch 10/10
6250/6250 [==============================] 

In [ ]:
model.save('/content/drive/MyDrive/1_INEL/Inel_ML/miModelo_texto_clasificacion',save_format='tf')

In [ ]:
x_train[-1]

'súper bien las brochas son buenas no sé menos han caído pelitos quizás pesan un poco pero con los chulas que son da igual xd'

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/1_INEL/Inel_ML/miModelo_texto_clasificacion')

In [ ]:
 loaded_model.predict(['no me gusta el libro de guillermo'])

1/1 [==============================] - 0s 376ms/step


array([[-2.4286082]], dtype=float32)